In [1]:
import torch
from telecom.scenario import Scenario
from rl.agent import Agent
import json
from dotdic import DotDic

In [2]:
with open('test_configs/config_debug_sce.json', 'r') as f:
    sce = DotDic(json.load(f))

with open('test_configs/config_debug_opt.json', 'r') as f:
    opt = DotDic(json.load(f))

In [3]:
scenario = Scenario(sce)

In [4]:
agents = []   # Vector of agents
nagents = opt.nagents
for i in range(10):
	agents.append(Agent(opt, sce, scenario, index=i, device='cpu'))

In [5]:
action = torch.zeros(nagents,dtype=int)
reward = torch.zeros(nagents)
QoS = torch.zeros(nagents)
state_target = torch.ones(nagents)

In [6]:
state = torch.zeros(nagents)  # Reset the state   
next_state = torch.zeros(nagents)  # Reset the next_state
nstep = 0
nepisode = 0
episode_return = 0.0  # Cumulative reward (return)
total_reward = 0.0  # Sum of rewards for average calculation
reward_count = 0
eps_min = 0.0
eps_increment = 0.003

In [7]:
eps_threshold = eps_min + eps_increment * nstep * (nepisode + 1)
eps_threshold 

0.0

In [8]:
for i in range(nagents):
    action[i] = agents[i].Select_Action(state, scenario, eps_threshold)

action

tensor([22, 18, 14,  1,  7,  7, 13,  1,  0, 11])

In [11]:
action = torch.tensor([ 4,  6, 22, 23, 23,  2, 21,  9, 22, 10])

In [12]:
for i in range(nagents):
    QoS[i], reward[i], _ = agents[i].Get_Reward(action, action[i], state, scenario)  # Obtain reward and next state
    next_state[i] = QoS[i]
    episode_return += reward[i].item()
    total_reward += reward[i].item()
    reward_count += 1

print(f'QoS: {QoS}\n Reward: {reward}')

QoS: tensor([1., 1., 0., 0., 0., 1., 0., 0., 0., 0.])
 Reward: tensor([ 0.9996,  1.0000, -0.5000, -0.5000, -0.5000,  1.0000, -0.5000, -0.5000,
        -0.5000, -0.5000])
